In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np 
import random 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

from IPython.display import Image 
from IPython.core.display import HTML 
    
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library


Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url).text

In [3]:
from bs4 import SoupStrainer

data = BeautifulSoup(response, "lxml")

In [4]:
def is_the_only_string_within_a_tag(s):
    return (s == s.parent.string)

data1 = data.find_all(string=is_the_only_string_within_a_tag)

In [5]:
data2 = data1[29:893]

In [6]:
print(len(data2))

864


In [7]:
PostCode = data2[0:862:3]
Borough = data2[1:863:3]
Neighborhood = data2[2:864:3]

In [8]:
import pandas as pd
d = {'PostCode': PostCode, 'Borough': Borough, 'Neighborhood': Neighborhood}
df = pd.DataFrame(data = d)

In [9]:
data1 = df.loc[df['Borough'] != 'Not assigned']

In [10]:
data1.head(10)
data1 = data1.reset_index(drop = True)

In [11]:
#data1['Neighborhood'] = data1['Neighborhood'].str.replace('Not assigned', 'Queen"s Park')

In [12]:
data1.to_csv('Table')

In [13]:
data = pd.read_csv('Table')

In [14]:
data['Neighborhood'] = data['Neighborhood'].str.replace('Not assigned', "Queen's Park").str.replace('\n', '')
data.head(11)

,Unnamed: 0,PostCode,Borough,Neighborhood
0,0,M3A,North York,Parkwoods
1,1,M4A,North York,Victoria Village
2,2,M5A,Downtown Toronto,Harbourfront
3,3,M5A,Downtown Toronto,Regent Park
4,4,M6A,North York,Lawrence Heights
5,5,M6A,North York,Lawrence Manor
6,6,M7A,Queen's Park,Queen's Park
7,7,M9A,Etobicoke,Islington Avenue
8,8,M1B,Scarborough,Rouge
9,9,M1B,Scarborough,Malvern


In [15]:
data.drop('Unnamed: 0', axis = 1, inplace = True)

In [16]:
data.head()

,PostCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [17]:
data.shape

(211, 3)

In [18]:
url = 'http://cocl.us/Geospatial_data'
coordinates = pd.read_csv(url)

In [19]:
coordinates.tail(20)

,Postal Code,Latitude,Longitude
83,M6R,43.648960,-79.456325
84,M6S,43.651571,-79.484450
85,M7A,43.662301,-79.389494
86,M7R,43.636966,-79.615819
87,M7Y,43.662744,-79.321558
88,M8V,43.605647,-79.501321
89,M8W,43.602414,-79.543484
90,M8X,43.653654,-79.506944
91,M8Y,43.636258,-79.498509
92,M8Z,43.628841,-79.520999


In [64]:
data1 = data.sort_values('PostCode')

In [65]:
data1['Latitude'] = coordinates.Latitude
data1['Longitude'] = coordinates.Longitude

In [66]:
data1.reset_index(inplace=True)

In [67]:
data1.drop('index', axis = 1, inplace=True)

In [68]:
data1['Latitude'] = data1['Latitude'].replace(np.nan, 0)
data1['Longitude'] = data1['Longitude'].replace(np.nan, 0)

In [69]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [70]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

In [78]:
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = map_toronto
kclusters = 211
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
clusters_for=data1['Neighborhood']
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(data1['Latitude'], data1['Longitude'], data1['Neighborhood'], clusters_for):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[len(cluster)-1],
        fill=True,
        fill_color=rainbow[len(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters